## Run this cell and copy the output into a shell terminal on the target server.  It will append an ssh public key the authorized_keys file for the user of that shell terminal:

In [ ]:
echo "cat >>~/.ssh/authorized_keys <<EOF"
cat ~/.ssh/id_rsa.pub
echo "EOF"

## Edit the ssh command in the following cell so that it will log into the target host:

In [ ]:
extra="-q -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null"
SSHCMD="ssh edit-this@10.0.3.2 $extra"

## Run this cell to test that the key was successfully appended ssh information is correct:

In [ ]:
$SSHCMD hostname || echo "ssh is not set up correctly"

## Edit the target directory on the target Server
All files for the build will go in this directory.  The repository will be copied to \$TARGETDIR/jupyter-platform-dev/.
The actual build directory will be at \$TARGETDIR/installdir/.

In [ ]:
TARGETDIR=/edit/this/path

## Edit to give the name of the bridge to which tap devices should be attached.

In [ ]:
BRIDGENAME='test-br0'

## Run the next cell to write out the above information to a config file.

In [ ]:
report_problem()
{
    echo "$*"
    exit 1 # exit the subshell below
}

check_ssh()
{
    [ "$(echo $SSHCMD)" != "" ] || report_problem "SSHCMD is not set"
    [ "$($SSHCMD echo OK)" = "OK" ] || report_problem "SSHCMD ($SSHCMD) did not connect successfully"
}

check_targetdir()
{
    $SSHCMD "[ -d '$TARGETDIR' ]" || report_problem "The directory ($TARGETDIR) does not exist on the server"
}

check_bridge()
{
    [ "$(echo $BRIDGENAME)" != "" ] || report_problem "BRIDGENAME is not set"
    [[ "$($SSHCMD /sbin/brctl show "$BRIDGENAME" 2>&1 1>/dev/null)" == "" ]] || report_problem "bridge ($BRIDGENAME) not found"
}

( # subprocess so error code will not exit jupyter's bash kernel
    check_ssh
    check_targetdir
    check_bridge
    set -e
    cat >~/jh_test.config <<EOF
SSHCMD="$SSHCMD"
TARGETDIR="$TARGETDIR"
BRIDGENAME="$BRIDGENAME"
EOF
    echo "Config created successfully"
)